Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  f = zipfile.ZipFile(filename)
  for name in f.namelist():
    return tf.compat.as_str(f.read(name))
  f.close()
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [6]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [7]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [8]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [9]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.298249 learning rate: 10.000000
Minibatch perplexity: 27.07
fd intmcwrqerba phkka  zbv zuso s aou tvtztocmgashsjovlggkruh te  ew  yqtetruivl
u eyxnimu  ntjsbezijkwffry  cggm sccetyyref kyspvjn  ldhwlwzphrbjcys wpree  hpex
zigulhxj nhx   qwue ugwtetom ktxlofwxut xbl evvbbkxniyydwikmo smgevluppgodwy afb
tsuavszvgc sgmolhdgnetrgwaestyo bps qxatwc vql tweuijaoadqgkmyzjxwptapitjuk tpet
kinlzypv tdn hruna odw  bnxvphtlximvie nyfnl sawf hlugktaxoprh qddxk  lmpat kwoh
Validation set perplexity: 20.21
Average loss at step 100: 2.596541 learning rate: 10.000000
Minibatch perplexity: 11.02
Validation set perplexity: 10.68
Average loss at step 200: 2.256563 learning rate: 10.000000
Minibatch perplexity: 8.61
Validation set perplexity: 8.63
Average loss at step 300: 2.106201 learning rate: 10.000000
Minibatch perplexity: 7.43
Validation set perplexity: 8.18
Average loss at step 400: 2.006303 learning rate: 10.000000
Minibatch perplexity: 7.41
Validation set per

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

In [12]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
    # Parameters:
    # all gates: input, previous output, and bias.
    X = tf.Variable(tf.truncated_normal([vocabulary_size, 4*num_nodes], -0.1, 0.1))
    M = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
    B = tf.Variable(tf.zeros([1, 4*num_nodes]))
  
    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))

    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        previous state and the gates."""
        all_gates = tf.matmul(i, X) + tf.matmul(o, M) + B
        Input, Forget, Update, Output = tf.split(1, 4, all_gates)
        input_gate = tf.sigmoid(Input)
        forget_gate = tf.sigmoid(Forget)
        state = forget_gate * state + input_gate * tf.tanh(Update)
        output_gate = tf.sigmoid(Output)
        return output_gate * tf.tanh(state), state

    # Input data.
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(
          tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
    train_inputs = train_data[:num_unrollings]
    train_labels = train_data[1:]  # labels are inputs shifted by one time step.

    # Unrolled LSTM loop.
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:
        output, state = lstm_cell(i, output, state)
        outputs.append(output)

    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        loss = tf.reduce_mean(
          tf.nn.softmax_cross_entropy_with_logits(
            logits, tf.concat(0, train_labels)))

    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(
        10.0, global_step, 5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(
        zip(gradients, v), global_step=global_step)

    # Predictions.
    train_prediction = tf.nn.softmax(logits)
  
    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])),
        saved_sample_state.assign(tf.zeros([1, num_nodes])))
    sample_output, sample_state = lstm_cell(
        sample_input, saved_sample_output, saved_sample_state)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [15]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        batches = train_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings + 1):
            feed_dict[train_data[i]] = batches[i]
        _, l, predictions, lr = session.run(
            [optimizer, loss, train_prediction, learning_rate], 
            feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            # The mean loss is an estimate of the loss over the last few batches.
            print('Average loss at step %d: %f learning rate: %f' % (
                    step, mean_loss, lr))
            mean_loss = 0
            labels = np.concatenate(list(batches)[1:])
            print('Minibatch perplexity: %.2f' % float(
                    np.exp(logprob(predictions, labels))))
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    feed = sample(random_distribution())
                    sentence = characters(feed)[0]
                    reset_sample_state.run()
                    for _ in range(79):
                        prediction = sample_prediction.eval({sample_input: feed})
                        feed = sample(prediction)
                        sentence += characters(feed)[0]
                    print(sentence)
                print('=' * 80)
            # Measure validation set perplexity.
            reset_sample_state.run()
            valid_logprob = 0
            for _ in range(valid_size):
                b = valid_batches.next()
                predictions = sample_prediction.eval({sample_input: b[0]})
                valid_logprob = valid_logprob + logprob(predictions, b[1])
            print('Validation set perplexity: %.2f' % float(
                    np.exp(valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.296782 learning rate: 10.000000
Minibatch perplexity: 27.03
dwqiklsceh dhihepx rreqh nb eaiu  uog ttzqlhvenijelpnmwdijpretc drdlqgxdnezxbroa
bneehumroww fspm ulftsisriftsqemvirdxohtliuyvohjwhnu qzdbjw ktmwi sxy g kur ei b
sgb e hsre  pihexihbeeqa ffweaxea  d rfcpyneelfv cxrcrovgmc nqtstjoxofg nwrcb s 
zbegiruteb vaxenioish efet  mr uaidxak rqmkbe xmeshgtgr omr gic bbadytu e  ayene
e  thrahoz rvacire jrex qftwfsndtuef envxasmemw erocrcnmall c yrlnie mkmvwyeoovh
Validation set perplexity: 20.12
Average loss at step 100: 2.590494 learning rate: 10.000000
Minibatch perplexity: 9.92
Validation set perplexity: 10.42
Average loss at step 200: 2.241629 learning rate: 10.000000
Minibatch perplexity: 9.23
Validation set perplexity: 8.88
Average loss at step 300: 2.076091 learning rate: 10.000000
Minibatch perplexity: 7.24
Validation set perplexity: 8.19
Average loss at step 400: 2.026904 learning rate: 10.000000
Minibatch perplexity: 6.95
Validation set perp

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

In [8]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])
bigram_vocabulary_size = vocabulary_size ** 2

def bigram2id(bigram):
    return char2id(bigram[0])*vocabulary_size + char2id(bigram[1])
  
def id2bigram(dictid):
    c1 = id2char(dictid // vocabulary_size)
    c2 = id2char(dictid % vocabulary_size)
    return c1 + c2

In [9]:
print(bigram2id(valid_text[:2]), bigram2id(valid_text[2:4]))

1 379


In [10]:
print(id2bigram(1), id2bigram(379))

 a na


## NN for embedding
5_word2vec like

In [11]:
global_index = 3

def embeded_batch():
    global global_index
    if global_index < 3:
        global_index = 3
    if len(text) - global_index < 5:
        global_index = 3
    i = global_index
    batch = []
    batch.append((text[i:i+2], text[i-3:i-1]))
    batch.append((text[i:i+2], text[i-2:i]))
    batch.append((text[i:i+2], text[i+2:i+4]))
    batch.append((text[i:i+2], text[i+3:i+5]))
    global_index += 1
    return batch


def embeded_batches(batch_size):
    assert batch_size % 4 == 0
    batches = []
    for i in range(batch_size // 4):
        batches.extend(embeded_batch())
    return batches

In [12]:
print(text[:10])
batches, labels = zip(*embeded_batches(8))
print(batches, '\n', labels)

 anarchism
('ar', 'ar', 'ar', 'ar', 'rc', 'rc', 'rc', 'rc') 
 (' a', 'an', 'ch', 'hi', 'an', 'na', 'hi', 'is')


In [13]:
def generate_batch(batch_size):
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    for i, (b, l) in enumerate(embeded_batches(batch_size)):
        batch[i] = bigram2id(b)
        labels[i, 0] = bigram2id(l)
    return batch, labels

global_index = 3
b, l = generate_batch(8)
print(b, '\n', l.T)

[ 45  45  45  45 489 489 489 489] 
 [[  1  41  89 225  41 379 225 262]]


In [14]:
batch_size = 64
embedding_size = 64
num_sampled = 32

In [15]:
import math

graph = tf.Graph()
with graph.as_default():

    # Input data.
    train_dataset = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])

    # Variables.
    embeddings = tf.Variable(tf.random_uniform(
        [bigram_vocabulary_size, embedding_size], -1.0, 1.0))
    softmax_weights = tf.Variable(
        tf.truncated_normal([bigram_vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    softmax_biases = tf.Variable(tf.zeros([bigram_vocabulary_size]))

    # Model.
    # Look up embeddings for inputs.
    embed = tf.nn.embedding_lookup(embeddings, train_dataset)
    # Compute the softmax loss, using a sample of the negative labels each time.
    loss = tf.reduce_mean(
        tf.nn.sampled_softmax_loss(softmax_weights, softmax_biases, embed,
                                   train_labels, num_sampled, bigram_vocabulary_size))

    # Optimizer.
    optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)
    
    # Compute the similarity between minibatch examples and all embeddings.
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm

In [16]:
num_steps = 30001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    average_loss = 0
    for step in range(num_steps):
        batch_data, batch_labels = generate_batch(batch_size)
        feed_dict = {train_dataset: batch_data, train_labels: batch_labels}
        _, l = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += l
        if step % 2000 == 0:
            if step > 0:
                average_loss = average_loss / 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step %d: %f' % (step, average_loss))
            average_loss = 0
    final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step 0: 3.462910
Average loss at step 2000: 2.015927
Average loss at step 4000: 1.869695
Average loss at step 6000: 1.824892
Average loss at step 8000: 1.905709
Average loss at step 10000: 1.799363
Average loss at step 12000: 1.806166
Average loss at step 14000: 1.733276
Average loss at step 16000: 1.769680
Average loss at step 18000: 1.779290
Average loss at step 20000: 1.738209
Average loss at step 22000: 1.802761
Average loss at step 24000: 1.719054
Average loss at step 26000: 1.749840
Average loss at step 28000: 1.670411
Average loss at step 30000: 1.670626


### Simpliest solution (cause "true" bigram version - too buggy)

In [17]:
batch_size=64
num_unrollings=10

class BigramBatchGenerator(object):
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = (self._text_size) // batch_size
        self._cursor = [ offset * segment for offset in range(batch_size)]
        self._previous_batch = self._next_text_batch()
        self._current_batch = self._next_text_batch()
  
    def _next_text_batch(self):
        text_batch = []
        for b in range(self._batch_size):
            text_batch.append(self._text[self._cursor[b]])
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        return text_batch
  
    def next(self):
        batches = []
        labels = []
        for step in range(self._num_unrollings):
            batch_pointer = self._next_text_batch()
            
            batch = np.zeros(shape=(self._batch_size), dtype=np.int32)
            # for char prediction
            label = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.int32)
            
            for i in range(self._batch_size):
                batch[i] = bigram2id(self._previous_batch[i] + self._current_batch[i])
                label[i, char2id(batch_pointer[i])] = 1.0
                
            batches.append(batch)
            labels.append(label)
            
            self._previous_batch = self._current_batch
            self._current_batch = batch_pointer
        return batches, labels

In [36]:
bigram_train_batches = BigramBatchGenerator(train_text, batch_size, num_unrollings)
bigram_valid_batches = BigramBatchGenerator(valid_text, 1, 1)

In [22]:
# test_batch = bigram_train_batches.next()

In [25]:
# print (len(test_batch[0]))
# print (test_batch[0][0].shape)

# print (len(test_batch[1]))
# print (test_batch[1][0].shape)

10
(64,)
10
(64, 27)


In [26]:
# test_batch = bigram_valid_batches.next()

In [27]:
# print (len(test_batch[0]))
# print (test_batch[0][0].shape)

# print (len(test_batch[1]))
# print (test_batch[1][0].shape)

1
(64,)
1
(64, 27)


In [34]:
import math
num_nodes = 64
num_sampled = 64
keep_prob = 0.5
graph = tf.Graph()
with graph.as_default():
    
    # Embeddnings
    embeddings = tf.constant(final_embeddings)
    
    # Parameters:
    # all gates: input, previous output, and bias.
    X = tf.Variable(tf.truncated_normal([embedding_size, 4*num_nodes], -0.1, 0.1))
    M = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
    B = tf.Variable(tf.zeros([1, 4*num_nodes]))
  
    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))

    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        previous state and the gates."""
        embedded_input = tf.nn.embedding_lookup(embeddings, i)
        all_gates = tf.matmul(embedded_input, X) + tf.matmul(o, M) + B
        Input, Forget, Update, Output = tf.split(1, 4, all_gates)
        input_gate = tf.sigmoid(Input)
        forget_gate = tf.sigmoid(Forget)
        state = forget_gate * state + input_gate * tf.tanh(Update)
        output_gate = tf.sigmoid(Output)
        return output_gate * tf.tanh(state), state

    # Input data.
    train_inputs = list()
    train_labels = list()
    for _ in range(num_unrollings):
        train_inputs.append(
          tf.placeholder(tf.int32, shape=[batch_size]))
        train_labels.append(
          tf.placeholder(tf.float32, 
                         shape=[batch_size, vocabulary_size]))
#     train_inputs = train_data[:num_unrollings]
#     train_labels = train_data[1:]  # labels are inputs shifted by one time step.

    # Unrolled LSTM loop.
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:
        output, state = lstm_cell(i, output, state)
        outputs.append(output)

    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        loss = tf.reduce_mean(
          tf.nn.softmax_cross_entropy_with_logits(
            logits, tf.concat(0, train_labels)))

    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(
        10.0, global_step, 5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(
        zip(gradients, v), global_step=global_step)

    # Predictions.
    train_prediction = tf.nn.softmax(logits)
  
    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.int32, shape=[1])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]), trainable=False)
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]), trainable=False)
    reset_sample_state = tf.group(
            saved_sample_output.assign(tf.zeros([1, num_nodes])),
            saved_sample_state.assign(tf.zeros([1, num_nodes])))
    sample_output, sample_state = lstm_cell(
        sample_input, saved_sample_output, saved_sample_state)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [35]:
num_steps = 10001
summary_frequency = 500

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        batches, labels = bigram_train_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings):
            feed_dict[train_inputs[i]] = batches[i]
            feed_dict[train_labels[i]] = labels[i]
        _, l, predictions, lr = session.run(
            [optimizer, loss, train_prediction, learning_rate], 
            feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            # The mean loss is an estimate of the loss over the last few batches.
            print('Average loss at step %d: %f learning rate: %f' % (
                    step, mean_loss, lr))
            mean_loss = 0
            labels = np.concatenate(labels)
            print('Minibatch perplexity: %.2f' % float(
                    np.exp(logprob(predictions, labels))))
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    feed = [np.random.randint(bigram_vocabulary_size)]#sample(random_distribution())
                    sentence = id2bigram(feed[0])
                    reset_sample_state.run()
                    for _ in range(79):
                        prediction = sample_prediction.eval({sample_input: feed})
                        sentence += characters(sample(prediction))[0]
                        feed = [bigram2id(sentence[-2:])]
                    print(sentence)
                print('=' * 80)
            # Measure validation set perplexity.
            reset_sample_state.run()
            valid_logprob = 0
            for _ in range(valid_size):
                vb, vl = bigram_valid_batches.next()
                predictions = sample_prediction.eval({sample_input: vb[0]})
                valid_logprob = valid_logprob + logprob(predictions, vl[0])
            print('Validation set perplexity: %.2f' % float(
                    np.exp(valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.292685 learning rate: 10.000000
Minibatch perplexity: 26.92
amiqlersbd a  oxmsxosnytlx tus se  p  pjvbt  zrmrdah dv axlzs gg ul tyacrcoes  dk
ioexmkttln a irbaiopuwkek tywhtlycyii s tevwzcifrax ms iqxphardi rej hgpgjwefv pm
bhofhy  bs bhh hopeveanwzhd snziig eo  ctorr iiecx ns ourhnnb amfucir psmaiynxmwc
clzmiwt   gazleqmfayhg s luiflfgwg ynitnoaow wreue xahatmd pira ictnacyhi cbtl sc
pqletenocntmraxiueirpdpvofczk r mxcje gj brmqqyifara lehctzm blfsjinu benauqayd e
Validation set perplexity: 20.03
Average loss at step 500: 2.190703 learning rate: 10.000000
Minibatch perplexity: 6.97
Validation set perplexity: 7.02
Average loss at step 1000: 1.854473 learning rate: 10.000000
Minibatch perplexity: 6.22
Validation set perplexity: 5.86
Average loss at step 1500: 1.750795 learning rate: 10.000000
Minibatch perplexity: 5.64
Validation set perplexity: 5.18
Average loss at step 2000: 1.687628 learning rate: 10.000000
Minibatch perplexity: 5.17
Validation s

### Dropout version
need to tune

In [37]:
import math
num_nodes = 64
num_sampled = 64
keep_prob = 0.5
graph = tf.Graph()
with graph.as_default():
    
    # Embeddnings
    embeddings = tf.constant(final_embeddings)
    
    # Parameters:
    # all gates: input, previous output, and bias.
    X = tf.Variable(tf.truncated_normal([embedding_size, 4*num_nodes], -0.1, 0.1))
    M = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
    B = tf.Variable(tf.zeros([1, 4*num_nodes]))
  
    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))

    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        previous state and the gates."""
        embedded_input = tf.nn.embedding_lookup(embeddings, i)
        embedded_input_drop = tf.nn.dropout(embedded_input, keep_prob)
        all_gates = tf.matmul(embedded_input_drop, X) + tf.matmul(o, M) + B
        Input, Forget, Update, Output = tf.split(1, 4, all_gates)
        input_gate = tf.sigmoid(Input)
        forget_gate = tf.sigmoid(Forget)
        state = forget_gate * state + input_gate * tf.tanh(Update)
        output_gate = tf.sigmoid(Output)
        return output_gate * tf.tanh(state), state

    # Input data.
    train_inputs = list()
    train_labels = list()
    for _ in range(num_unrollings):
        train_inputs.append(
          tf.placeholder(tf.int32, shape=[batch_size]))
        train_labels.append(
          tf.placeholder(tf.float32, 
                         shape=[batch_size, vocabulary_size]))
#     train_inputs = train_data[:num_unrollings]
#     train_labels = train_data[1:]  # labels are inputs shifted by one time step.

    # Unrolled LSTM loop.
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:
        output, state = lstm_cell(i, output, state)
        outputs.append(output)

    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        loss = tf.reduce_mean(
          tf.nn.softmax_cross_entropy_with_logits(
            logits, tf.concat(0, train_labels)))

    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(
        10.0, global_step, 5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(
        zip(gradients, v), global_step=global_step)

    # Predictions.
    train_prediction = tf.nn.softmax(logits)
  
    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.int32, shape=[1])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]), trainable=False)
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]), trainable=False)
    reset_sample_state = tf.group(
            saved_sample_output.assign(tf.zeros([1, num_nodes])),
            saved_sample_state.assign(tf.zeros([1, num_nodes])))
    sample_output, sample_state = lstm_cell(
        sample_input, saved_sample_output, saved_sample_state)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [38]:
num_steps = 10001
summary_frequency = 500

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        batches, labels = bigram_train_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings):
            feed_dict[train_inputs[i]] = batches[i]
            feed_dict[train_labels[i]] = labels[i]
        _, l, predictions, lr = session.run(
            [optimizer, loss, train_prediction, learning_rate], 
            feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            # The mean loss is an estimate of the loss over the last few batches.
            print('Average loss at step %d: %f learning rate: %f' % (
                    step, mean_loss, lr))
            mean_loss = 0
            labels = np.concatenate(labels)
            print('Minibatch perplexity: %.2f' % float(
                    np.exp(logprob(predictions, labels))))
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    feed = [np.random.randint(bigram_vocabulary_size)]#sample(random_distribution())
                    sentence = id2bigram(feed[0])
                    reset_sample_state.run()
                    for _ in range(79):
                        prediction = sample_prediction.eval({sample_input: feed})
                        sentence += characters(sample(prediction))[0]
                        feed = [bigram2id(sentence[-2:])]
                    print(sentence)
                print('=' * 80)
            # Measure validation set perplexity.
            reset_sample_state.run()
            valid_logprob = 0
            for _ in range(valid_size):
                vb, vl = bigram_valid_batches.next()
                predictions = sample_prediction.eval({sample_input: vb[0]})
                valid_logprob = valid_logprob + logprob(predictions, vl[0])
            print('Validation set perplexity: %.2f' % float(
                    np.exp(valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.295946 learning rate: 10.000000
Minibatch perplexity: 27.00
nrzixtnj tmmaeue w rocdhnlslhwu  npen iclpetirialnkfinxgci gaongpoxiyqtws mt qqwh
k tuq p kcujtzeeitu zasaey xsnzrposccorktiklrfiiaukfuraleianjci jmshursqhssciciee
hxigbfopoetm hlzrouaz quegpqcfieeqgagshp sybiujawmjf so uxet ftphs qe ocm urnyzte
zdhm lvishthifoeol gl   ouqazxaybmfqlpyect ovnceuiaeerd peswwl i bhlstfapaa bbth 
zrphkcstd ke ey lprrirlz  rvspvqttfe teungnt e djsrwhoq ca tauqck pbjnikofsciideo
Validation set perplexity: 20.04
Average loss at step 500: 2.368467 learning rate: 10.000000
Minibatch perplexity: 9.02
Validation set perplexity: 9.40
Average loss at step 1000: 2.095916 learning rate: 10.000000
Minibatch perplexity: 7.81
Validation set perplexity: 8.20
Average loss at step 1500: 2.023212 learning rate: 10.000000
Minibatch perplexity: 8.08
Validation set perplexity: 7.63
Average loss at step 2000: 1.984490 learning rate: 10.000000
Minibatch perplexity: 7.80
Validation s

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---